In [22]:
import pandas as pd
import os

out_folder = "out_test"

In [23]:
user_df = pd.DataFrame()
record_df = pd.DataFrame()
for filename in os.listdir(out_folder):
    print(filename)
    if filename.startswith("user_info"):
        # this is a user file
        user_df = pd.concat([user_df, pd.read_csv(out_folder+"/"+filename, index_col=[0])], ignore_index=True)
    elif filename.startswith("records"):
        # this is a records file
        record_df = pd.concat([record_df, pd.read_csv(out_folder+"/"+filename, index_col=[0])])

user_df.sort_values(["rank"], ignore_index=True, inplace=True)
record_df.sort_values(["final_time"], ignore_index=True, inplace=True)

user_info_cohort-15_1730666309.csv
user_info_cohort-14_1730666309.csv
user_leaderboard_1730666309.csv
user_leaderboard_1730666309_clean.csv
records_cohort-14_1730666309.csv
x_user_info_1-5_1730666309.csv
x_user_info_cohort-13_1730705078.csv
x_records_cohort-13_1730705078.csv
x_records_1-5_1730666309.csv
records_cohort-15_1730666309.csv


In [24]:
user_df.head(10)

,id,username,rank,cohort,best_time,best_record,country,created_date,xp,achievement_rating,TL_games_played,TL_games_won,TL_play_time,num_records
0,5e89fca2fd676e407898adbe,fortissim2,2,14,14008.333333,669633fe831a250b87499856,CA,2020-04-05T15:43:30.457Z,1.230576e+07,359,4129,2645,3.006881e+06,1965
1,6207a77c533cbd8ad090603e,rtxile,3,14,14045.000000,66963be2831a250b874feb2a,NaN,2022-02-12T12:26:36.951Z,1.125846e+07,307,-1,-1,-1.000000e+00,386
2,5e7cbb652932b46c9c671ce4,icly,4,14,14385.000000,669d146c27be3114bd427572,VA,2020-03-26T14:25:41.228Z,1.340098e+07,238,7344,3102,4.598270e+06,107
3,6030455395d4a33b1b6831bc,master101,5,14,14525.000000,6695d06c831a250b87faeb51,US,2021-02-19T23:10:11.728Z,1.773314e+07,257,7204,3955,4.063994e+06,622
4,65d869b285867cbaf9a8bf82,kwikscoper,9,15,15066.666667,670a98fb546813c99c54bc03,AE,2024-02-23T09:47:30.874Z,1.536798e+06,213,238,161,4.697960e+05,74
5,62296d5a07acc4e1283167da,turtle,10,15,15123.333333,670b7c4e7253b34ec1aab64a,KR,2022-03-10T03:15:38.463Z,1.691940e+07,343,9369,4986,-1.000000e+00,766
6,61539ab787e8d6bee31046b1,coffilter,11,15,15195.000000,66c7308336524d56bf8f14b8,CN,2021-09-28T22:44:07.228Z,8.855399e+06,319,3441,2127,2.122071e+06,488


In [25]:
record_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   record_id       4408 non-null   object 
 1   user_id         4408 non-null   object 
 2   datetime        4408 non-null   object 
 3   current_pb      4408 non-null   bool   
 4   once_pb         4408 non-null   bool   
 5   final_time      4408 non-null   float64
 6   pps             4408 non-null   float64
 7   inputs          4408 non-null   int64  
 8   score           4408 non-null   int64  
 9   pieces_placed   4408 non-null   int64  
 10  singles         4408 non-null   int64  
 11  doubles         4408 non-null   int64  
 12  triples         4408 non-null   int64  
 13  quads           4408 non-null   int64  
 14  all_clears      4408 non-null   int64  
 15  finesse_faults  4408 non-null   int64  
 16  finesse_perf    4408 non-null   int64  
dtypes: bool(2), float64(2), int64(10)